<a href="https://colab.research.google.com/github/ITanmayee/wise_ml/blob/main/CNN_Cats_and_dogs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importing the Keras libraries and packages
import os
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt


unzipping and loading the data set

In [ ]:
!unzip "<File Path of the cats_and_dogs_filtered.zip in mounted drive>" -d "/content"

In [ ]:
"""base_dir = os.path.join(os.getcwd(), 'cats_and_dogs_filtered')
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
"""
base_dir = '/content/drive/MyDrive/Colab Notebooks/datasets/cats_and_dogs_filtered/cats_and_dogs_filtered'
train_dir = '/content/drive/MyDrive/Colab Notebooks/datasets/cats_and_dogs_filtered/train'
validation_dir = '/content/drive/MyDrive/Colab Notebooks/datasets/cats_and_dogs_filtered/validation'

In [ ]:
"""train_cats_dir = os.path.join(train_dir, 'cats')  # directory with our training cat pictures
train_dogs_dir = os.path.join(train_dir, 'dogs')  # directory with our training dog pictures
validation_cats_dir = os.path.join(validation_dir, 'cats')  # directory with our validation cat pictures
validation_dogs_dir = os.path.join(validation_dir, 'dogs')  # directory with our validation dog pictures"""

train_cats_dir = '/content/drive/MyDrive/Colab Notebooks/datasets/cats_and_dogs_filtered/train/cats' # directory with our training cat pictures
train_dogs_dir = '/content/drive/MyDrive/Colab Notebooks/datasets/cats_and_dogs_filtered/train/dogs' # directory with our training dog pictures
validation_cats_dir = '/content/drive/MyDrive/Colab Notebooks/datasets/cats_and_dogs_filtered/validation/cats'  # directory with our validation cat pictures
validation_dogs_dir = '/content/drive/MyDrive/Colab Notebooks/datasets/cats_and_dogs_filtered/validation/dogs'  # directory with our validation dog pictures


In [ ]:
num_cats_tr = len(os.listdir(train_cats_dir))
num_dogs_tr = len(os.listdir(train_dogs_dir))

num_cats_val = len(os.listdir(validation_cats_dir))
num_dogs_val = len(os.listdir(validation_dogs_dir))

total_train = num_cats_tr + num_dogs_tr
total_val = num_cats_val + num_dogs_val

Setting Model Parameters

In [ ]:
BATCH_SIZE = 100  # Number of training examples to process before updating our models variables
IMG_SHAPE  = 150  # Our training data consists of images with width of 150 pixels and height of 150 pixels

Model Creation

In [ ]:
# Initialising the CNN
classifier = Sequential()

In [ ]:
# Step 1 - Convolution
classifier.add(Conv2D(32, (3, 3), input_shape = (150, 150, 3), activation = 'relu'))

In [ ]:
# Step 2 - Pooling
classifier.add(MaxPooling2D(2, 2))

In [ ]:
# Step 1 & 2 with more hidden layers
classifier.add(Conv2D(64, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(2,2))

classifier.add(Conv2D(128, (3,3), activation='relu'))
classifier.add(MaxPooling2D(2,2))

classifier.add(Conv2D(128, (3,3), activation='relu'))
classifier.add(MaxPooling2D(2,2))

In [ ]:
# Step 3 - Dropout
classifier.add(Dropout(0.5))

Adding a second convolutional layer classifier.add(Conv2D(32, (3, 3), activation = 'relu')) classifier.add(MaxPooling2D(pool_size = (2, 2)))

In [ ]:
# Step 4 - Flattening
classifier.add(Flatten())

In [ ]:
# Step 5 - Full connection
classifier.add(Dense(units = 512, activation = 'relu'))
classifier.add(Dense(units = 2, activation = 'softmax'))

In [ ]:
# Compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
classifier.summary()

Data Augmentation and Preparation

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

In [ ]:
training_set = train_datagen.flow_from_directory(batch_size=BATCH_SIZE,
                                                 directory='/content/drive/MyDrive/Colab Notebooks/datasets/cats_and_dogs_filtered/train',
                                                 shuffle=True,
                                                 target_size=(IMG_SHAPE,IMG_SHAPE),
                                                 class_mode='binary')

In [ ]:
validation_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
validation_set = validation_datagen.flow_from_directory(batch_size=BATCH_SIZE,
                                                        directory='/content/drive/MyDrive/Colab Notebooks/datasets/cats_and_dogs_filtered/validation',
                                                        target_size=(IMG_SHAPE, IMG_SHAPE),
                                                        class_mode='binary')

Train the model

In [ ]:
epochs=100
history = classifier.fit_generator(
    training_set,
    steps_per_epoch=int(np.ceil(total_train / float(BATCH_SIZE))),
    epochs=epochs,
    validation_data=validation_set,
    validation_steps=int(np.ceil(total_val / float(BATCH_SIZE)))
)

Training Result Visualization

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()